In [0]:
read_json = spark.read.json('dbfs:/databricks-datasets/samples/people/people.json')
display(read_json)

In [0]:
#display(dbutils.fs.ls('/databricks-datasets/samples'))
display(dbutils.fs.ls('dbfs:/databricks-datasets/samples/newsgroups/20_newsgroups'))